In [1]:
import os
import vedo
import numpy as np
import trimesh
from copy import copy
from scipy.special import sph_harm
import ipywidgets as widgets
from ipywidgets import interact
import pyvista as pv
from IPython import embed
import re

# Spherical harmonics

The spherical harmonics are defined by the following formula

$$Y_{lm}(\theta, \phi)=\sqrt{\frac{2l+1}{4\pi}}P_l^m(\cos\theta)e^{im\phi},$$

where the $P_l^m$ are the associated Legendre polynomials. They satisfy the following equation:

$$r^2\nabla^2Y_{lm}(\theta, \phi)=l(l+1)Y_{lm}(\theta, \phi)$$

In other words, the Laplacian operator $\nabla^2$ can be diagonalized by using this basis.


The deformation is performed along the radial direction at each point, and with an amplitude given by the following expansion:

$$ \textbf{d}(\theta,\phi,t) =\bigg( \sum_{l=0}^{l_{\text{max}}}\sum_{m=-l}^{l}a_{lm}(t)Y_{lm}(\theta, \phi)\bigg) \hat{\textbf{r}}$$

$$a_{lm}(t)=b_{lm}^{(0)}+\sum_{n=1}^{N_t}a_{lm}^{(n)}\sin(2\pi nt)+b_{lm}^{(n)}\cos(2\pi nt),\,\, t\in[0,1]$$

### A local deformation

The expansion of the Dirac delta is given by:

$$\delta(\theta,\phi)=\sum_{l=0}^{\infty}\sum_{m=-l}^{l}Y_{lm}(0, 0)Y_{lm}(\theta, \phi)$$

Would this be useful to create a local deformation (by truncating this expansion)?

### How to perform content and style deformations

One possibility is to generate a basis of spherical harmonics that spans the subspace of possible content deformations; and another that spans the subspace of style deformations.

$$ \textbf{d}_c(\theta,\phi)\in span(\mathcal{B}_c)$$
$$ \textbf{d}_s(\theta,\phi)\in span(\mathcal{B}_s)\subseteq span(\mathcal{B}_c)^{\bot}$$

The style deformation is such that it varies with time in a periodical manner. Therefore, the overall deformation will be of the form

$$ \textbf{d}(\theta,\phi,t) = \textbf{d}_c(\theta,\phi) + A(t)\textbf{d}_s(\theta,\phi) $$ with $$A(t)=A(t+2\pi)$$


For example, we can make $$A(t)=A_0\sin t$$

The generative process is such that:

$$a_{lm}^{(n)}\sim\mathcal{N}(0,1)$$

for each $l$, $m$ and $n$. These $a_{lm}^{(n)}$ are the latent variables of the model.

### Election of content and style subspaces

Let's call $E_l$ the eigenspaces of the laplacian with eigenvalues $l(l+1)$ and let's make, arbitrarily

$$\mathcal{S}_c=E_1\oplus E_2$$
$$\mathcal{S}_s=E_3\oplus E_4$$

where $\oplus$ represents the direct sum of vector spaces.

_____________

# <span style="color:red">Development</span>

$$a_{lm}(t)=b_{lm}^{(0)}+\sum_{n=1}^{N}a_{lm}^{(n)}\sin(2\pi nt)+b_{lm}^{(n)}\cos(2\pi nt)$$

We cache 
$$f_{lmn}(\theta,\phi,t)=Y_{lm}(\theta,\phi)\sin(2\pi nt)$$
$$g_{lmn}(\theta,\phi,t)=Y_{lm}(\theta,\phi)\cos(2\pi nt)$$

Sample coefficients $a_{lm}^{(n)}\sim \mathcal{N}(0,\sigma^2)$.


### Generate population

We build a dictionary  $Y_{lm}(\theta, \phi)$ for all the $(\theta, \phi)$ in the discretization of $S_2$. First we get the coordinates of such discretization:

In [28]:
from utils import cache_base_functions, generate_population

amplitude_static_max = 0.1
amplitude_dynamic_max = 0.05
N = 100; T = 20
freq_max = 2; l_max = 2
mesh_resolution = 12

#sphere = vedo.Sphere(res=mesh_resolution).to_trimesh()
#sphere_coords = appendSpherical_np(sphere.vertices)[:,3:]
# Y_lm, f_lmn, g_lmn = cache_base_functions(sphere_coords, l_max, freq_max, Nt=T)

avg_meshes, moving_meshes, coefs = generate_population(
    N, T, l_max, freq_max,
    amplitude_static_max, 
    amplitude_dynamic_max,     
    mesh_resolution,
    random_seed=1
)

In [36]:
moving_meshes[0][i].vertices

TrackedArray([[ 0.00000000e+00,  0.00000000e+00, -1.11068509e+00],
              [ 0.00000000e+00,  0.00000000e+00,  1.11068509e+00],
              [-1.98904898e-16, -1.08278779e+00, -1.55681380e-01],
              [-1.98904898e-16, -1.08278779e+00,  1.55681380e-01],
              [-1.86681112e-16, -1.01624458e+00,  4.64103371e-01],
              [-1.86681112e-16, -1.01624458e+00, -4.64103371e-01],
              [-1.11575479e-16, -6.07388615e-01, -9.45115094e-01],
              [-1.11575479e-16, -6.07388615e-01,  9.45115094e-01],
              [-5.79480667e-17, -3.15454581e-01, -1.07433964e+00],
              [-5.79480667e-17, -3.15454581e-01,  1.07433964e+00],
              [-1.56151905e-16, -8.50051429e-01, -7.36573748e-01],
              [-1.56151905e-16, -8.50051429e-01,  7.36573748e-01],
              [ 4.80572584e-17,  7.84834597e-01, -6.80063043e-01],
              [ 4.80572584e-17,  7.84834597e-01,  6.80063043e-01],
              [ 1.88090247e-17,  3.07174672e-01, -1.04614086e+

In [48]:
avg_meshes[0].vertices.

TrackedArray([[ 0.00000000e+00,  0.00000000e+00, -1.13716936e+00],
              [ 0.00000000e+00,  0.00000000e+00,  1.13716936e+00],
              [-1.99011405e-16, -1.08336758e+00, -1.55764742e-01],
              [-1.99011405e-16, -1.08336758e+00,  1.55764742e-01],
              [-1.89110977e-16, -1.02947215e+00,  4.70144201e-01],
              [-1.89110977e-16, -1.02947215e+00, -4.70144201e-01],
              [-1.14825652e-16, -6.25081733e-01, -9.72646122e-01],
              [-1.14825652e-16, -6.25081733e-01,  9.72646122e-01],
              [-5.96366796e-17, -3.24646961e-01, -1.10564601e+00],
              [-5.96366796e-17, -3.24646961e-01,  1.10564601e+00],
              [-1.59781741e-16, -8.69811336e-01, -7.53695805e-01],
              [-1.59781741e-16, -8.69811336e-01,  7.53695805e-01],
              [ 4.81348863e-17,  7.86102358e-01, -6.81161563e-01],
              [ 4.81348863e-17,  7.86102358e-01,  6.81161563e-01],
              [ 1.91059087e-17,  3.12023156e-01, -1.06265328e+

In [62]:
np.sqrt((moving_meshes[0][i].vertices - avg_meshes[0].vertices)**2).sum(axis=1).mean()

TrackedArray(0.02510852)

In [41]:
for i in range(20):
  print("  ", np.mean(np.sqrt(np.sum((moving_meshes[0][i].vertices - avg_meshes[0].vertices)**2, axis=1))))

   0.031543453398200236
   0.043031260335863686
   0.05035616342299152
   0.05276688846148033
   0.0500306776501678
   0.04244726452815052
   0.030777270873863626
   0.016131675199724277
   0.002429472839131369
   0.01700811634855912
   0.031543453398200236
   0.04303126033586367
   0.05035616342299152
   0.05276688846148034
   0.050030677650167804
   0.04244726452815053
   0.030777270873863637
   0.01613167519972428
   0.0024294728391313723
   0.01700811634855912


In [29]:
conn = vedo.Sphere(res=mesh_resolution).to_trimesh().faces
conn = np.c_[np.ones(conn.shape[0]) * 3, conn].astype(int)  # add column of 3

def f(i, average):
    
    if average:
        mesh = pv.PolyData(avg_meshes[1].vertices, conn)
    else:
        mesh = pv.PolyData(moving_meshes[1][i-1].vertices, conn)
    pl = pv.Plotter(notebook=True, off_screen=False, polygon_smoothing=False)
    pl.add_mesh(mesh, show_edges=True)
    pl.show(use_panel=True, interactive=True, interactive_update=True)

In [30]:
#interact(f, i=widgets.IntSlider(min=1,max=N))
interact(f, i=widgets.IntSlider(min=1,max=T), average=widgets.ToggleButton());

interactive(children=(IntSlider(value=1, description='i', max=20, min=1), ToggleButton(value=False, descriptio…

### Mesh visualization with Pyvista

In [6]:
from gif import generate_gif_population

In [7]:
# connectivity
conn = vedo.Sphere(res=mesh_resolution).to_trimesh().faces
conn = np.c_[3 * np.ones(conn.shape[0]), conn].astype(int)  # add column of 3 to make it PyVista-compatible

generate_gif_population(moving_meshes, mesh_connectivity=conn)

In [8]:
kk = np.array([
  np.array([np.array(timeframe.vertices) for timeframe in individual])
  for individual in population
])

import pickle as pkl

with open("synthetic_population.pkl", "wb") as ff:
    pkl.dump({"population_meshes": kk, "coefficients": coefs}, ff)

NameError: name 'population' is not defined

# Interactive 3D mesh plotting

In [ ]:
def apply_deformation(sphere, spherical_coords, amplitude_lm):
    
    deformed_sphere = sphere
    
    coefs = np.ones(spherical_coords.shape[0])
    
    for l, m in amplitude_lm:        
        coefs += amplitude_lm[l, m] * Y_lm[l,m]
                
    deformed_sphere.vertices = np.multiply(
        deformed_sphere.vertices, np.kron(np.ones((3, 1)), coefs).transpose()
    )
    
    return deformed_sphere


In [ ]:
def get_coef_as_string(l,m):
    return "a" + str(l) + str(m).replace("-","_")

def get_lm(string):
    import re    
    regex = re.compile(".*([0-9])(_?[0-9])")
    l = int(regex.match(string)[1])
    m = int(regex.match(string)[2].replace("_","-"))
    return l,m    

In [ ]:
conn = vedo.Sphere().to_trimesh().faces
conn = np.c_[np.ones(conn.shape[0]) * 3, conn].astype(int)  # add column of 3

def f(**kwargs):

    coefs = {get_lm(lm):kwargs[lm] for lm in kwargs}
    
    sphere = vedo.Sphere().to_trimesh()
    spherical_coords = _appendSpherical_np(sphere.vertices)[:,3:]
    deformed_sphere = apply_deformation(sphere, spherical_coords, coefs)
        
    mesh = pv.PolyData(deformed_sphere.vertices, conn)
    pl = pv.Plotter(notebook=True, off_screen=False, polygon_smoothing=True)
    pl.add_mesh(mesh, show_edges=True)
    pl.show(use_panel=True, interactive=True, interactive_update=True)

In [ ]:
coefs = {get_coef_as_string(*lm): widgets.FloatSlider(min=-0.5, max=0.5) for lm in indices}
interact(f, **coefs);

In [ ]:
# t = 15
# mesh_list[t].show(faces=True)

In [ ]:
#def f(l, m, amplitude):
#    return DeformedSphere().deform(amplitude, l, m).show()

#### Saving the synthetic meshes as `obj` files

In [ ]:
# odir = "/home/rodrigo/tmp/mallas_obj"
# os.makedirs(odir, exist_ok=True)
# 
# N = 100
# 
# for i in range(N):
#     for t, mesh in enumerate(mesh_list):
#         
#         # print(mesh.vertices)
#         j = str(j)
#         j = "0" * (2 - len(j)) + j # 5 --> 05, 15 --> 15
#         
#         with open(os.path.join(odir, "sphere_{}.obj".format(i)), "wt") as ff:        
#             obj_content = trimesh.exchange.export.export_obj(
#                 mesh, 
#                 include_normals=False, 
#                 include_texture=False
#             )
#             ff.write(obj_content)
#             
#         # print(mesh.show())

__________________________

# Attempt to visualize meshes (to complete)

#### `itkwidgets`

In [ ]:
import itk
import itkwidgets
from itkwidgets import view

#### `Trimesh`

In [ ]:
import trimesh

#### `pyvista`

#### `polyscope`

In [ ]:
import polyscope as ps
ps.init()
mesh = mesh_list[25]
ps.register_surface_mesh("sph", mesh.vertices, mesh.faces, smooth_shade=True)
ps.show()

________________________

In [ ]:
DeformedSphere().deform(0, 0, 0).vertices

In [ ]:
interact(f, t=widgets.IntSlider(N_t-1, max=N_t-1));

In [ ]:
# np.polynomial.chebyshev.Chebyshev((1,2,3))()